In [1]:
import random
from datetime import datetime
from helper import generate_random_date, generate_random_image, hash_password
from dictionary import *
import mysql.connector

In [2]:
def gen_enjoyer(id):
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    full_name = f"{first_name} {last_name}"
    avatar = generate_random_image(256, random.randint(3,6))
    username = first_name.lower() + last_name.lower()[0] + "{:04d}".format(random.randint(0, 9999))
    password = hash_password(username[::-1])
    email = username + random.choice(EMAIL_SUFFIX)
    bio = f"I love {random.choice(GENRES)} music."
    enjoyment = round(random.random()*10, 2)
    return ([id, username, password, full_name, avatar, email, bio], [id, enjoyment]) # (User, Enjoyer)

def gen_artist(id):
    first_name = random.choice(FIRST_NAMES)
    last_name = random.choice(LAST_NAMES)
    full_name = f"{first_name} {last_name}"
    avatar = generate_random_image(256, random.randint(3,6))
    username = first_name.lower() + last_name.lower()[0] + "{:04d}".format(random.randint(0, 9999))
    password = hash_password(username[::-1])
    email = username + random.choice(EMAIL_SUFFIX)
    bio = f"I'm a musician from {random.choice(LOCATIONS)}!"
    verified = random.choice([True, False])
    return ([id, username, password,full_name, avatar, email, bio], [id, verified]) # (User, Artist)

def gen_album(id):
    name = f"{random.choice(ALBUM_1)} {random.choice(ALBUM_2)} {random.choice(ALBUM_3)}"
    creation_date = generate_random_date()
    cover_art = generate_random_image(256, random.randint(3,6))
    return ([id, name, creation_date], [id, cover_art])

def gen_playlist(id, enjoyer_id):
    name = f"{random.choice(PLAYLIST_1)} {random.choice(PLAYLIST_2)}"
    creation_date = generate_random_date()
    description = f"{random.choice(PLAYLIST_DESC_PREFIX)} {random.choice(PLAYLIST_DESC_SUFFIX)}"
    cover_art = generate_random_image(256, random.randint(3,6))
    return [id, name, creation_date], [id, enjoyer_id,  description, cover_art]

def gen_track(id, album_id, album_genre, album_date):
    p = random.random()
    if p<0.05:
        name = f"{random.choice(SONG_1)} {random.choice(SONG_2)} {random.choice(SONG_3)}"
    elif p<0.90:
        name = f"{random.choice(SONG_1)} {random.choice(SONG_2)}"
    else:
        name = f"{random.choice(SONG_1)}"
    p = random.random()
    if p<0.02:
        creation_date = generate_random_date(end_date=album_date)
    else:
        creation_date = album_date
    genre = album_genre
    length_seconds = random.randint(60,600)
    return [id, name, creation_date], [id, album_id, genre, length_seconds]

def gen_playlist_tracks(playlist_id, track_id):
    return [playlist_id, track_id]

def gen_performs_in(artist_id, track_id, plays):
    return [artist_id, track_id, plays]

def gen_friend(friend_id1, friend_id2):
    since = generate_random_date(datetime(2000, 1, 1))
    return [friend_id1, friend_id2, since]

def gen_reaction(user_id, content_id, txt_type):
    p = random.random()
    if p<0.33:
        txt = f"{random.choice(TXT_1)} {random.choice(TXT_2)} {random.choice(txt_type)}"
        emoji = None
    elif p<0.66:
        txt = None
        emoji = random.choice(EMOJIS)
    else:
        txt = f"{random.choice(TXT_1)} {random.choice(TXT_2)} {random.choice(txt_type)}"
        emoji = random.choice(EMOJIS)
    return [user_id, content_id, txt, emoji]

def gen_follows(enjoyer_id, artist_id):
    return [enjoyer_id, artist_id]

def gen_saved(enjoyer_id, content_id):
    return [enjoyer_id, content_id]
   

In [3]:
N_ENJOYER = 1000
N_ARTIST = 100
N_ALBUM = 200
N_TRACKS_ALBUM = (5, 15)
N_PLAYLIST = 1000
N_TRACKS_PLAYLIST = (0, 40)
N_FRIENDS_PER_USER = (0,20)
N_REACTION_PER_USER  = (0,25)  
N_REACTION_PER_ARTIST  = (0,5) 
N_FOLLOWS_PER_USER = (0,15)
N_SAVED_PER_USER = (0,50)

users = []
enjoyers = []
for i in range(0,N_ENJOYER):
    user_enjoyer = gen_enjoyer(i)
    users.append(user_enjoyer[0])
    enjoyers.append(user_enjoyer[1])
artists = []
for i in range(N_ENJOYER, N_ENJOYER+N_ARTIST):
    user_artist = gen_artist(i)
    users.append(user_artist[0])
    artists.append(user_artist[1])

contents = []
albums = []
for i in range(0,N_ALBUM):
    content_album = gen_album(i)
    contents.append(content_album[0])
    albums.append(content_album[1])

playlists = []
for i in range(N_ALBUM, N_ALBUM+N_PLAYLIST):
    content_playlist = gen_playlist(i,random.choice(enjoyers)[0])
    contents.append(content_playlist[0])
    playlists.append(content_playlist[1])


content_i = len(albums) + len(playlists)
tracks = []
performs_in = []
for album in albums:
    artist = random.choice(artists)
    instrument = random.choice(INSTRUMENTS)
    album_genre = random.choice(GENRES)
    for i in range(0, random.randint(*N_TRACKS_ALBUM)):
        content_track = gen_track(content_i, album[0], album_genre, contents[album[0]][2])
        contents.append(content_track[0])
        tracks.append(content_track[1])
        performs_in.append(gen_performs_in(artist[0], content_i, instrument))
        for i in range(0, random.randint(0,5)):
            if random.random() < 0.5:
                performs_in.append(gen_performs_in(random.choice(artists)[0], content_i, random.choice(INSTRUMENTS)))
        content_i += 1

playlist_tracks = []
for playlist in playlists:
    for i in range(0, random.randint(*N_TRACKS_PLAYLIST)):
        playlist_tracks.append(gen_playlist_tracks(playlist[0], random.choice(tracks)[0]))

friends = []
users_A = users[0:int((N_ENJOYER+N_ARTIST)/2)]
users_B = users[int((N_ENJOYER+N_ARTIST)/2):(N_ENJOYER+N_ARTIST)]
for a in users_A:
    for i in range(0, random.randint(*N_FRIENDS_PER_USER)):
        friends.append(gen_friend(a[0], random.choice(users_B)[0]))

reactions = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_REACTION_PER_USER)):
        p = random.random()
        if p<0.33:
            content_id = random.choice(albums)[0]
            txt_type = TXT_ALBUM
        elif p<0.66:
            content_id = random.choice(playlists)[0]
            txt_type = TXT_PLAYLIST
        else:
            content_id = random.choice(tracks)[0]
            txt_type = TXT_SONG
        reactions.append(gen_reaction(enjoyer[0], content_id, txt_type))

for artist in artists:
    for i in range(0, random.randint(*N_REACTION_PER_ARTIST)):
        p = random.random()
        if p<0.33:
            content_id = random.choice(albums)[0]
            txt_type = TXT_ALBUM
        elif p<0.66:
            content_id = random.choice(playlists)[0]
            txt_type = TXT_PLAYLIST
        else:
            content_id = random.choice(tracks)[0]
            txt_type = TXT_SONG
        reactions.append(gen_reaction(artist[0], content_id, txt_type))

follows = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_FOLLOWS_PER_USER)):
        follows.append(gen_follows(enjoyer[0], random.choice(artists)[0]))

saves = []
for enjoyer in enjoyers:
    for i in range(0, random.randint(*N_SAVED_PER_USER)):
        saves.append(gen_follows(enjoyer[0], random.choice(albums+tracks+playlists)[0]))
     

In [4]:
output_dir = 'msn.sql'
objects = { "User": users,
            "Enjoyer": enjoyers,
            "Artist": artists,
            "Content": contents,
            "Album": albums,
            "Playlist": playlists,
            "Track": tracks,
            "PerformsIn": performs_in,
            "PlaylistTracks": playlist_tracks,
            "Friend": friends,
            "Reaction": reactions,
            "Follows": follows,
            "Saved": saves}

def get_insert_query(table: str, *params):
    params_str = ""
    for p in params:
        if p == None:
            params_str += "NULL"
        else:
            if isinstance(p, int):
                params_str += str(p)
            elif isinstance(p, str):
                params_str += f'"{str(p)}"'
            elif isinstance(p, bool):
                params_str += str(p)
        params_str += ","
    return f"INSERT IGNORE INTO `{table}` VALUES ({params_str[:-1]});\n"

connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="phanchi99", 
  auth_plugin='mysql_native_password'
)
cursor = connection.cursor(buffered=True)
cursor.execute("USE msn")
for (table, entries) in objects.items():
    for entry in entries:
        if table == "User":
            values = {
                'id': entry[0],
                'username': entry[1],
                'password': entry[2],
                'full_name': entry[3],
                'avatar': entry[4],
                'email': entry[5],
                'bio': entry[6]
            }
            query = "INSERT IGNORE INTO User (id, username, password, full_name, avatar, email, bio) " \
                    "VALUES (%(id)s, %(username)s, %(password)s, %(full_name)s, %(avatar)s, %(email)s, %(bio)s)"
            cursor.execute(query, values)
        elif table == "Enjoyer":
            values = {
                'id': entry[0],
                'enjoyment': entry[1]
            }
            query = "INSERT IGNORE INTO Enjoyer (id, enjoyment) " \
                    "VALUES (%(id)s, %(enjoyment)s)"
            cursor.execute(query, values)

        elif table == "Artist":
            values = {
                'id': entry[0],
                'verified': entry[1]
            }
            query = "INSERT IGNORE INTO Artist (id, verified) " \
                    "VALUES (%(id)s, %(verified)s)"
            cursor.execute(query, values)

        elif table == "Content":
            values = {
                'id': entry[0],
                'name': entry[1],
                'creation_date': entry[2]
            }
            query = "INSERT IGNORE INTO Content (id, name, creation_date) " \
                    "VALUES (%(id)s, %(name)s, %(creation_date)s)"
            cursor.execute(query, values)

        elif table == "Album":
            values = {
                'id': entry[0],
                'cover_art': entry[1]
            }
            query = "INSERT IGNORE INTO Album (id, cover_art) " \
                    "VALUES (%(id)s, %(cover_art)s)"
            cursor.execute(query, values)

        elif table == "Playlist":
            values = {
                'id': entry[0],
                'enjoyer_id': entry[1],
                'description': entry[2],
                'cover_art': entry[3]
            }
            query = "INSERT IGNORE INTO Playlist (id, enjoyer_id, description, cover_art) " \
                    "VALUES (%(id)s, %(enjoyer_id)s, %(description)s, %(cover_art)s)"
            cursor.execute(query, values)

        elif table == "Track":
            values = {
                'id': entry[0],
                'album_id': entry[1],
                'genre': entry[2],
                'length_seconds': entry[3]
            }
            query = "INSERT IGNORE INTO Track (id, album_id, genre, length_seconds) " \
                    "VALUES (%(id)s, %(album_id)s, %(genre)s, %(length_seconds)s)"
            cursor.execute(query, values)

        elif table == "PerformsIn":
            values = {
                'artist_id': entry[0],
                'track_id': entry[1],
                'plays': entry[2]
            }
            query = "INSERT IGNORE INTO PerformsIn (artist_id, track_id, plays) " \
                    "VALUES (%(artist_id)s, %(track_id)s, %(plays)s)"
            cursor.execute(query, values)

        elif table == "PlaylistTracks":
            values = {
                'playlist_id': entry[0],
                'track_id': entry[1]
            }
            query = "INSERT IGNORE INTO PlaylistTracks (playlist_id, track_id) " \
                    "VALUES (%(playlist_id)s, %(track_id)s)"
            cursor.execute(query, values)

        elif table == "Friend":
            values = {
                'friend_id1': entry[0],
                'friend_id2': entry[1],
                'since': entry[2]
            }
            query = "INSERT IGNORE INTO Friend (friend_id1, friend_id2, since) " \
                    "VALUES (%(friend_id1)s, %(friend_id2)s, %(since)s)"
            cursor.execute(query, values)

        elif table == "Reaction":
            values = {
                'user_id': entry[0],
                'content_id': entry[1],
                'txt': entry[2],
                'emoji': entry[3]
            }
            query = "INSERT IGNORE INTO Reaction (user_id, content_id, txt, emoji) " \
                    "VALUES (%(user_id)s, %(content_id)s, %(txt)s, %(emoji)s)"
            cursor.execute(query, values)

        elif table == "Follows":
            values = {
                'enjoyer_id': entry[0],
                'artist_id': entry[1]
            }
            query = "INSERT IGNORE INTO Follows (enjoyer_id, artist_id) " \
                    "VALUES (%(enjoyer_id)s, %(artist_id)s)"
            cursor.execute(query, values)

        elif table == "Saved":
            values = {
                'enjoyer_id': entry[0],
                'content_id': entry[1]
            }
            query = "INSERT IGNORE INTO Saved (enjoyer_id, content_id) " \
                    "VALUES (%(enjoyer_id)s, %(content_id)s)"
            cursor.execute(query, values)
connection.commit()
cursor.close()
connection.close()